In [6]:
import FedTools as ft
import tokenisation as tkn
import pandas as pd
import api_calls as api
import os
import json
import running_model as rnm
import numpy as np
from IPython.display import display, clear_output

In [7]:
minutes_raw = pd.read_parquet("./data/scraping/all_minutes.parquet")["Federal_Reserve_Mins"]

In [8]:
with open("./data/prompts/prompt_1.txt", "r") as file:
    prompt_1 = file.read().splitlines()

In [9]:
minutes_raw_filtered = minutes_raw[minutes_raw.apply(lambda x: len(x) <= 150_000)]

In [10]:
for i in range(len(minutes_raw)):
    clear_output(wait=True)
    date = minutes_raw.index[i].strftime("%Y%m%d")
    if os.path.exists(f"./data/results/prompt_1/gpt-3.5-turbo/minutes/gpt-3.5-turbo_prompt_1_fomcminutes{date}.json"):
        display(date)
        continue
    display(f"date: {minutes_raw.index[i]}, progress: {round(i / len(minutes_raw), 2)}%")
    result = []
    tokenised = [ele for ele in tkn.basic_tokeniser(minutes_raw.iloc[i]) if len(ele) < 250]
    sentences = tkn.get_important_text(tokenised)[:1000]
    raw_sentiments = await api.get_multiple_api_calls("gpt-3.5-turbo", prompt_1, sentences)
    labels = [rnm._find_label(sentiment[1]) for sentiment in raw_sentiments]
    reasons = [sentiment[0] for sentiment in raw_sentiments]
    for sentence, label, reason in zip(sentences, labels, reasons):
        result.append({"sentence": sentence, "label": label, "reason" : reason})
    with open(f"./data/results/prompt_1/gpt-3.5-turbo/minutes/gpt-3.5-turbo_prompt_1_fomcminutes{date}.json", "w") as file:
        json.dump(result, file)

'date: 2003-12-09 00:00:00, progress: 0.35%'

2023-10-17 12:39:49 [info     ] completed 12 / 1               index=12 response_status=200 total=1
2023-10-17 12:39:49 [info     ] completed 18 / 1               index=18 response_status=200 total=1
2023-10-17 12:39:49 [info     ] completed 2 / 1                index=2 response_status=200 total=1
2023-10-17 12:39:49 [info     ] completed 16 / 1               index=16 response_status=200 total=1
2023-10-17 12:39:49 [info     ] completed 1 / 1                index=1 response_status=200 total=1
2023-10-17 12:39:49 [info     ] completed 39 / 1               index=39 response_status=200 total=1
2023-10-17 12:39:49 [info     ] completed 31 / 1               index=31 response_status=200 total=1
2023-10-17 12:39:49 [info     ] completed 44 / 1               index=44 response_status=200 total=1
2023-10-17 12:39:49 [info     ] completed 10 / 1               index=10 response_status=200 total=1
2023-10-17 12:39:49 [info     ] completed 33 / 1               index=33 response_status=200 total=1
20

ConnectionAbortedError: Too many errors. Aborted API Call

In [13]:
with open("test.txt", "w") as file:
    file.write(minutes_raw.loc["2020-11-05"])

In [15]:
tokenised = [ele for ele in tkn.basic_tokeniser(minutes_raw.loc["2020-11-05"]) if len(ele) < 250]
sentences = tkn.get_important_text(tokenised)

In [16]:
len(sentences)

195